In [ ]:
def transfer_learning(output_folder_name, input_function_name, output_function_name, num_days, mins_before_predict, window_slide,  patient_name, trained_model_name, monitor, learning_rate, epochs, batch_size, seed_number):    
    output_folder_results = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(output_function_name)+'/results'
    os.makedirs(output_folder_results, exist_ok=True) 
    output_folder_models = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(output_function_name)+'/models'
    os.makedirs(output_folder_models, exist_ok=True) 
    trained_folder_models = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(input_function_name)+'/models'
    
    random.seed(seed_number) 
    np.random.seed(seed_number)
    tf.random.set_seed(seed_number)

        
    directory = os.getcwd()

    file_path = os.path.join(directory, patient_name)
    print(file_path)
    df = pd.read_csv(file_path)
    df = df[0:480*num_days] ###########use 5 days datat only

    RMSE = []
    
    
    
    df_training = df.iloc[:int(480*(num_days-2))]
    df_validation = df.iloc[int(480*(num_days-2)):int(480*(num_days-1))]
    df_testing = df.iloc[int(480*(num_days-1)):480*num_days]


    # Extract the input features and target variable
    X_training = df_training[['insulin', 'CHO', 'CGM']]
    y_training = df_training['CGM']

    X_validation = df_validation[['insulin', 'CHO', 'CGM']]
    y_validation = df_validation['CGM']

    X_testing = df_testing[['insulin', 'CHO', 'CGM']]
    y_testing = df_testing['CGM']
    # Convert the DataFrame to numpy arrays
    X_training= X_training.to_numpy()
    y_training = y_training.to_numpy()

    X_validation= X_validation.to_numpy()
    y_validation = y_validation.to_numpy()

    X_testing= X_testing.to_numpy()
    y_testing = y_testing.to_numpy()


    results_df = pd.DataFrame(columns=['Round', 'File', 'RMSE_global','RMSE_local'])    

    # Create input sequences and corresponding target values
    X_sequences_training = []
    y_targets_training = []

    X_sequences_validation = []
    y_targets_validation = []

    X_sequences_testing = []
    y_targets_testing = []

    
    timesteps = int(mins_before_predict/3)
    
    for i in range(0, len(X_training) - timesteps-window_slide):
        X_sequences_training.append(X_training[i:i+window_slide])
        y_targets_training.append(y_training[i+timesteps+window_slide-1])

    # Convert sequences and targets to numpy arrays
    X_sequences_training = np.array(X_sequences_training)
    y_targets_training = np.array(y_targets_training)

    for i in range(0, len(X_validation) - timesteps-window_slide):
        X_sequences_validation.append(X_validation[i:i+window_slide])
        y_targets_validation.append(y_validation[i+timesteps+window_slide-1])

    # Convert sequences and targets to numpy arrays
    X_sequences_validation = np.array(X_sequences_validation)
    y_targets_validation = np.array(y_targets_validation)


    for i in range(0, len(X_testing) - timesteps-window_slide):
        X_sequences_testing.append(X_testing[i:i+window_slide])
        y_targets_testing.append(y_testing[i+timesteps+window_slide-1])

    # Convert sequences and targets to numpy arrays
    X_sequences_testing = np.array(X_sequences_testing)
    y_targets_testing = np.array(y_targets_testing)
    


    filepath_trained_model = os.path.join(trained_folder_models, trained_model_name)
    local_model = load_model(filepath_trained_model)
    output_model_name = 'TL_'+str(num_days)+'_'+str(patient_name)+'_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+'.h5'
    output_model_filepath = os.path.join(output_folder_models, output_model_name)

    checkpointer = ModelCheckpoint(filepath=output_model_filepath, verbose=1, save_best_only=True, save_weights_only=False, monitor=monitor)
    early_stopper = EarlyStopping(monitor=monitor, mode='auto', patience=5)
    optimizer = Adam(learning_rate=learning_rate)
    local_model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    history = local_model.fit(X_sequences_training, y_targets_training, batch_size=batch_size, epochs = epochs, verbose=1, validation_data=(X_sequences_validation, y_targets_validation), callbacks=[checkpointer, early_stopper])

    local_model = load_model(output_model_filepath)                    
    y_pred = local_model.predict(X_sequences_testing)
    RMSE_values_local = np.sqrt(mean_squared_error(y_targets_testing, y_pred))
    print('RMSE value of local model: %.4f' %RMSE_values_local)
    return RMSE_values_local